<a href="https://colab.research.google.com/github/sarawoldeyohannes/BigData/blob/main/Big_Data_Programming_Assignment_3_v_viii.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get -qq update
!apt-get -qq install -y openjdk-11-jdk-headless
!pip install -q pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .appName("BigDataAssignment3-Friends")
    .master("local[*]")
    .config("spark.ui.port", "4050")
    .getOrCreate()
)
sc = spark.sparkContext

!wget -q 'https://drive.google.com/uc?export=download&id=1GlLc9fJlvwMUzDNmXP2s38a7cSBKEcFT' -O 'Friends.txt'


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package java-common.
(Reading database ... 125082 files and directories currently installed.)
Preparing to unpack .../java-common_0.72build2_all.deb ...
Unpacking java-common (0.72build2) ...
Selecting previously unselected package libpcsclite1:amd64.
Preparing to unpack .../libpcsclite1_1.9.5-3ubuntu1_amd64.deb ...
Unpacking libpcsclite1:amd64 (1.9.5-3ubuntu1) ...
Selecting previously unselected package openjdk-11-jre-headless:amd64.
Preparing to unpack .../openjdk-11-jre-headless_11.0.28+6-1ubuntu1~22.04.1_amd64.deb ...
Unpacking openjdk-11-jre-headless:amd64 (11.0.28+6-1ubuntu1~22.04.1) ...
Selecting previously unselected package ca-certificates-java.
Preparing to unpack .../ca-certificates-java_20190909ubuntu1.2_all.deb ...
Unpacking ca-certificates-java (20190909u

In [ ]:
def txtSave(rdd, out_dir):
    import shutil, os
    if os.path.exists(out_dir):
        shutil.rmtree(out_dir)
    if hasattr(rdd, "coalesce"):
        rdd.coalesce(1, shuffle=False).saveAsTextFile(out_dir)
    else:
        sc.parallelize([str(rdd)], 1).saveAsTextFile(out_dir)

In [ ]:
# Part V
from itertools import combinations
from pyspark import StorageLevel

def parse_line(line):
    parts = line.strip().split('\t')
    u = int(parts[0])
    if len(parts) == 1 or parts[1].strip() == "":
        return (u, [])
    return (u, [int(x) for x in parts[1].split(',') if x.strip() != ""])

adj = sc.textFile("Friends.txt").map(parse_line).persist(StorageLevel.MEMORY_AND_DISK)
degrees = adj.map(lambda kv: (kv[0], len(kv[1]))).persist(StorageLevel.MEMORY_AND_DISK)
edges = (adj.flatMap(lambda kv: [((min(kv[0], f), max(kv[0], f)), 1) for f in kv[1]]).distinct()
           .persist(StorageLevel.MEMORY_AND_DISK))
all_users_kv = adj.map(lambda kv: (kv[0], None)).distinct().persist(StorageLevel.MEMORY_AND_DISK)


In [ ]:
#People you might know via the number of mutual friends

pcm = adj.flatMap(
    lambda kv: [((min(a,b), max(a,b)), 1) for (a,b) in combinations(kv[1], 2)]
)
# total mutual friends per pair
mutualcounts = pcm.reduceByKey(lambda a,b: a+b)
# remove direct friends
candidatepairs = mutualcounts.subtractByKey(edges)

dmutual = candidatepairs.flatMap(
    lambda kv: [(kv[0][0], (kv[0][1], kv[1])),
                (kv[0][1], (kv[0][0], kv[1]))]
)
top10mutual = (dmutual
                .groupByKey()
                .mapValues(lambda it: sorted(list(it), key=lambda x: (-x[1], x[0]))[:10]))

user_to_list_mutual = top10mutual.map(lambda kv: (kv[0], ",".join(str(cid) for cid,_ in kv[1])))
#line per user
mutual_lines = (all_users_kv
                .leftOuterJoin(user_to_list_mutual)
                .map(lambda kv: f"{kv[0]}\t{(kv[1][1] or '')}")
                .sortBy(lambda s: int(s.split('\t')[0])))

txtSave(mutual_lines, "outputBymutual")
print("Wrote outputBymutual")

Wrote outputBymutual


In [ ]:
#People you might know via the influence of a common friend

adjwdeg = adj.leftOuterJoin(degrees).mapValues(lambda t: (t[0], t[1] if t[1] is not None else 0))
paircontin = adjwdeg.flatMap(
    lambda kv: [((min(a,b), max(a,b)), 1.0/kv[1][1] if kv[1][1] > 0 else 0.0)
                for (a,b) in combinations(kv[1][0], 2)]
)

pairinf = paircontin.reduceByKey(lambda a,b: a+b)

candidatepairs = pairinf.subtractByKey(edges)

dinfl = candidatepairs.flatMap(
    lambda kv: [(kv[0][0], (kv[0][1], kv[1])),
                (kv[0][1], (kv[0][0], kv[1]))]
)
top10infl = (dinfl
              .groupByKey()
              .mapValues(lambda it: sorted(list(it), key=lambda x: (-x[1], x[0]))[:10]))

user_to_list_infl = top10infl.map(lambda kv: (kv[0], ",".join(str(cid) for cid,_ in kv[1])))
#line per user
infl_lines = (all_users_kv
              .leftOuterJoin(user_to_list_infl)
              .map(lambda kv: f"{kv[0]}\t{(kv[1][1] or '')}")
              .sortBy(lambda s: int(s.split('\t')[0])))

txtSave(infl_lines, "outputByinfluence")
print("Wrote outputByinfluence")


Wrote outputByinfluence


In [ ]:
# part vi (read me)
ids = [19, 231, 414, 512, 3972, 170000, 49400]
t = sc.parallelize(ids).map(lambda u: (u, ""))

mutfort = t.leftOuterJoin(top10mutual.map(lambda kv: (kv[0], ",".join(str(cid) for cid,_ in kv[1])))).map(lambda kv: (kv[0], kv[1][1] or ""))
inffort   = t.leftOuterJoin(top10infl.map(lambda kv: (kv[0], ",".join(str(cid) for cid,_ in kv[1])))).map(lambda kv: (kv[0], kv[1][1] or ""))

for uid, (m, i) in mutfort.join(inffort).sortByKey().collect():
    print(f"user {uid} -  Mutual= {m}   Influence= {i}")


user 19 -  Mutual= 14064,13800,13818,13967,1689,13795,13810,13898,14027,14043   Influence= 11378,14324,22555,13818,42697,14064,23556,44943,33051,17740
user 231 -  Mutual= 95   Influence= 95
user 414 -  Mutual= 411,413   Influence= 411,413
user 512 -  Mutual= 488,489,491,492,497,499,504,505,506,508   Influence= 44665,44712,44704,7573,17493,25288,31145,47842,488,489
user 3972 -  Mutual= 20841,22090,31128,33624,660,1100,1101,1595,1647,2989   Influence= 20841,33624,5353,16267,21780,23504,36754,47745,22090,31128
user 49400 -  Mutual= 15918,49398,49399   Influence= 15918,49398,49399
user 170000 -  Mutual=    Influence= 


In [ ]:
# part vii (read me)
ids = [
     86643545, 74197821
]

def last4(uid):
    s = str(uid)
    return int(s[-4:]) if len(s) >= 4 else int(s)

drv = [last4(u) for u in ids]
print("Derived IDs", drv if drv else "(none)")

dd = sc.parallelize(drv).map(lambda u: (u, ""))

mu = dd.leftOuterJoin(top10mutual.map(lambda kv: (kv[0], ",".join(str(cid) for cid,_ in kv[1])))).map(lambda kv: (kv[0], kv[1][1] or ""))
inf   = dd.leftOuterJoin(top10infl  .map(lambda kv: (kv[0], ",".join(str(cid) for cid,_ in kv[1])))).map(lambda kv: (kv[0], kv[1][1] or ""))

for uid, (m, i) in mu.join(inf).sortByKey().collect():
    print(f"user {uid} - Mutual= {m}   Influence= {i}")

Derived IDs [3545, 7821]
user 3545 - Mutual= 3546,3451,3456,3505,3519,3525,13430,660,3463,3478   Influence= 550,27779,3546,3554,1113,13430,42097,1852,2027,2066
user 7821 - Mutual= 5262,5376,7712,33667,1220,1689,2684,2916,3377,3409   Influence= 7712,8613,5262,5376,3377,33667,4509,26086,30959,5772
